In [1]:
import numpy as np
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")

import pymysql
from sqlalchemy import create_engine
import json
import random
import datetime
from datetime import timedelta ,time, datetime
from dateutil.relativedelta import relativedelta
import re
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [2]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

In [3]:
def select_data(date, hour):
    sql1 = f''' -- 订单&风控信息  近10日数据   
            SELECT om.create_time,om.id as order_id ,om.order_number,om.status, date(om.create_time) as create_date
            ,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
            when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
            when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
            ,tod.sku_attributes,tod.product_name,tod.new_actual_money
            ,om.user_mobile,tmu.true_name,tmu.id_card_num
            ,top.total_describes
            ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
            ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
            ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
            ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
            ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
            ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
            ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
            ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result
            ,cc.name as channel_name         -- 来源渠道
            ,cc.channel_type_id              -- 渠道id
            ,pa.name as activity_name        -- 活动名称
            ,om.merchant_id,om.merchant_name,pa.type
            ,om.order_method, om.activity_id
            , om.order_type, tor.update_time, tomt.reason, tpmn.name 机型, cc.scene, tp.id 商品ID, tp.name phone_name
            from  db_digua_business.t_order  om
            left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
            left join db_digua_business.t_order_risk tor on om.id = tor.order_id
            -- 备注信息合并 
            left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
            on om.id = top.order_id 
            -- 渠道名称
            left join db_digua_business.t_channel cc on om.channel = cc.scene 
            -- 活动名称
            left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
            -- 用户信息 
            left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
            -- 商品信息
            left join db_digua_business.t_order_details tod on om.id = tod.order_id
            -- 商家订单转移表
            left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id
            -- 商品表
            left join db_digua_business.t_product tp on tp.id = tod.product_id
            -- 商品型号
            left join db_digua_business.t_product_model_number tpmn on tpmn.id=tp.model_number_id
            where om.user_mobile is not null 
            and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
            ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
            and  date_format(om.create_time, '%Y-%m-%d')>='{date}'
            -- and date_format(om.create_time, '%Y-%m-%d')>='2025-07-01'
            -- and date_format(om.create_time, '%Y-%m-%d')<='2025-07-31'
            and hour(om.create_time)<'{hour}'
            '''
    df_order = clean.query(sql1)
    df_order = df_order[df_order.type != 4]
    sql_risk = ''' -- risk等级
                    select trace_id, id_card as id_card_r, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_r 
                    from db_credit.risk
                    '''
    df_risk = clean.query(sql_risk)
    sql3 = '''
                SELECT risk_trace_id trace_id, id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
                '''
    df_risk_examine = clean.query(sql3)
    df_re = df_risk_examine.copy()
    df_re.loc[:, 'time_re'] = pd.to_datetime(df_re.time.dt.date, errors="coerce")
    df_re.rename(columns={'id_card': 'id_card_re', 'status': 'status_re'}, inplace=True)

    sql_ra = ''' -- 996强拒表
                select order_id, time, replace(case when JSON_VALID(result) THEN JSON_EXTRACT(result, '$.status') end,'"','') as status_ra  from db_credit.risk_alipay_interactive_prod_result
                '''
    df_ra = clean.query(sql_ra)
    df_ra.loc[:, 'time_ra'] = pd.to_datetime(df_ra.time.dt.date, errors="coerce")
    df_ra = df_ra.sort_values(by='time', ascending=False).groupby('order_id').head(1)

    sql_upv = '''
    select
    tsc.scene, date(tsc.day) c_date, tsc.new_uv_count, tsc.uv_count, tsc.pv_count
    from db_digua_business.t_statistics_channel tsc
    '''
    df_upv = clean.query(sql_upv)
    df_order.loc[:, 'phone_name'] = df_order.phone_name.str.replace(' ', '').str.extract(r'(iPhone\d+(ProMax|Pro|Plus)?)')[0]
    df_order = df_order.merge(df_upv, left_on=['scene', 'create_date'], right_on=['scene', 'c_date'], how='left')
    df_order.loc[:,'商品ID'] = df_order.商品ID.astype(str)+'_'+df_order.phone_name


    return df_order, df_risk, df_risk_examine, df_re, df_ra

In [4]:
def clean_data(df, df_risk, df_re, df_ra, key=None):
    # 日期处理
    df["下单日期"] = df["create_time"].dt.date
    df["下单日期"] = pd.to_datetime(df["下单日期"], errors="coerce")
    df["月份"] = df["下单日期"].dt.month
    df["下单月份"] = df["create_time"].dt.strftime('%Y-%m')
    df['年份'] = df["下单日期"].dt.year
    df['hour'] = df['create_time'].dt.hour
    # 备注信息处理
    df['拒绝理由'] = df["rejected"].str.replace("[", "").str.replace("]", "").str.replace('"', '')
    df = df[df['sku_attributes'].notnull()]
    df["取消原因"] = df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
    df["电审拒绝原因"] = df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
    df["无法联系原因"] = df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()

    df.loc[:, "内存"] = df.apply(lambda x: clean.getneicun(x["sku_attributes"]), axis=1)
    df.loc[:, '机型内存'] = df.机型+'_'+df.内存
    df.loc[:, "进件"] = np.where((df["status2"] == "待支付") | (df["status2"] == "订单取消"), "未进件", "进件")
    df.loc[:, "来源渠道"] = df["channel_name"].fillna("未知渠道")
    # 渠道归属
    df.loc[:, "归属渠道"] = df.apply(
        lambda x: clean.qudao_type(x["来源渠道"], x["activity_name"], x["order_method"],
                                        x['channel_type_id'], x['order_type']), axis=1)
    # 京东不去重数据，不需要执行订单去重处理
    if key != '京东':
        # 订单去重处理
        df = clean.order_drop_duplicates(df)

    # 定义状态处理
    df = df.merge(df_risk[['trace_id', 'status_r']], on='trace_id', how='left').merge(
        df_re[['trace_id', 'status_re']], on='trace_id', how='left').merge(
        df_ra[['order_id', 'time_ra', 'status_ra']], left_on=['order_id', '下单日期'],
        right_on=['order_id', 'time_ra'], how='left')
    df['是否前置拦截'] = np.where(
        (df.result.str.contains('id_card不得为空')) | (df.result.str.contains('mobile校验不通过')) | (
            df.result.str.contains('name校验不通过'))
        | (df.result.str.contains('年龄超过49岁或低于18岁')) | (df.result == '风控未通过') | (
                df.result == '黑名单用户'), 1, 0)
    df['是否机审强拒'] = np.where(df.status_r == '1', 1, 0)
    df['是否出库前风控强拒'] = np.where((df.status_re == '1') | (df.status_ra == '1'), 1, 0)
    df.loc[:, "审核状态"] = df.apply(
        lambda x: clean.reject_type(x["拒绝理由"], x["进件"], x["电审拒绝原因"], x["取消原因"], x["status2"],
                                        x["无法联系原因"], x["total_describes"], x['是否前置拦截'], x['是否机审强拒'],x['是否出库前风控强拒']), axis=1)
    # 剔除商家数据
    df = clean.drop_merchant(df)
    # 获取节点状态数据
    df = clean.status_node(df)
    # 剔除据量数据
    df2 = df.copy()
    df2 = clean.drop_rejected_merchant(df2)
    df2 = df2[df2.是否拒量 == 0]

    return df, df2


In [5]:
def conversion_model(df, df2, df_risk_examine, model):
    today = datetime.now().strftime('%Y-%m-%d')
    # 排除二手手机
    df = df[df.是否二手==0]
    df2 = df2[df2.是否二手==0]
    # 获取机型的转化数据和uv数据
    def group(df1, df12):
        # 调用data_group函数，计算转化数据
        df_model_group = all_models.data_group(df1, df12, df_risk_examine, model)
        df_model_group.loc[:, '出库'] = df_model_group.出库+df_model_group.拒量出库
        df_model_group = df_model_group[["去重订单数", "前置拦截", "拦截率", "进件数", "预授权通过率", "机审强拒", "强拒比例", "机审通过件", "人审拒绝", "风控通过件", "风控通过率", "客户取消", "无法联系",
            "出库前风控强拒", "待审核", '出库',  '总体进件出库率（含拒量）','取消率', '人审拒绝率', '出库前强拒比例', '无法联系占比', '订单出库率']].fillna(0)
        # 判断是否是计算商品ID的转化数据，如不是则不需要添加uv数据
        if model == '商品ID':
            df1.loc[:, 'uv'] = np.where(df1.uv_count == 0, df1.new_uv_count, df1.uv_count)
            df1 = df1.drop_duplicates(subset=['order_id',"商品ID", "uv"])
            df_uv = df1.groupby('商品ID').agg({'uv': 'sum'})
            df_model_group = df_uv.merge(df_model_group, on=model, how='left')
        # 根据去重订单数进行倒序排序
        df_model_group = df_model_group.sort_values(by='去重订单数', ascending=False)
        return df_model_group
    # 当天
    # df_now = df[df.下单日期==today]
    # df2_now = df2[df2.下单日期==today]
    # df_model_group_now = group(df_now, df2_now)
    # # 近7天
    # df_7 = df[df.下单日期>=(datetime.now()-timedelta(days=7))]
    # df2_7 = df2[df2.下单日期>=(datetime.now()-timedelta(days=7))]
    # df_model_group_7 = group(df_7, df2_7)
    # 7天的每日数据不进行汇总
    df_model_group = group(df, df2)
    
    
    
    # # 近15天
    # df_model_group_15 = group(df, df2)
    return df_model_group


In [6]:
def run(date, hour):
    df, df_risk, df_risk_examine, df_re, df_ra = select_data(date, hour)
    return df, df_risk, df_risk_examine, df_re, df_ra

In [7]:
def my_job(hour, minute, hours):
    # 获取7天数据
    date = (datetime.now()-timedelta(days=7)).strftime('%Y-%m-%d')
    today = datetime.now().strftime('%Y%m%d')
    print(f'执行定时任务：现在是{today}的{hour}点{minute}分...')
    print('正在查询数据...')
    df, df_risk, df_risk_examine, df_re, df_ra = run(date, hours)
    print('数据查询完毕！\n正在清理数据...')
    df, df2 = clean_data(df, df_risk, df_re, df_ra)
    print('数据清理完毕！\n正在计算机型转化数据...')
    id_list = ['6710_iPhone16', '6711_iPhone16Plus', '6712_iPhone16Pro', '6713_iPhone16ProMax', '7681_iPhone16ProMax', '7682_iPhone16Pro', '7683_iPhone16Plus', '7684_iPhone16', '6752_iPhone16', '6756_iPhone16ProMax', '6757_iPhone16Pro']
    # 芝麻7天每日数据
    df_zm_uv = df[df.商品ID.isin(id_list[:-3])]
    df2_zm_uv = df2[df2.商品ID.isin(id_list[:-3])]
    df_model_group = conversion_model(df_zm_uv, df2_zm_uv, df_risk_examine, ['下单日期','商品ID'])
    print('商品ID转化数据计算完毕！')


    return df_model_group
    # with pd.ExcelWriter(path + f'芝麻商品近7日芝麻转化数据_商品ID_{today}.xlsx', engine='xlsxwriter') as writer:

    #         df_model_group.to_excel(writer, sheet_name='近7日商品ID转化')


In [8]:
today = datetime.now().strftime('%Y%m%d')
hour = 18
minute = 10
path = 'F:/需求/瑞彤需求/'
df_res = my_job(hour, minute, 18)



执行定时任务：现在是20250808的18点10分...
正在查询数据...


数据查询完毕！
正在清理数据...
数据清理完毕！
正在计算机型转化数据...
商品ID转化数据计算完毕！


In [9]:
# 按下单日期，商品ID升序排序
df_res = df_res.sort_values(['下单日期', '商品ID'], ascending=[True, True])
df_res.to_excel(path + f'近7日芝麻商品转化数据_商品ID_{today}.xlsx', index=True)